### yearly_album_chart에 있는 앨범들의 출시 날짜 크롤링

In [89]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

1. 검색명 생성

In [19]:
yearly_album_chart_df = pd.read_csv('data/yearly_album_chart_df.csv')
len(yearly_album_chart_df)

500

In [71]:
# 검색명 데이터프레임 만들기
search_list = []

for i in range(len(yearly_album_chart_df)):
    title_singer = list(yearly_album_chart_df.loc[i][2:4])
    search = ' '.join(title_singer)
    search_list.append(search)

len(search_list) # 500개 확인

500

In [72]:
search_name_df = pd.DataFrame()
search_name_df['search_name'] = search_list
len(search_name_df)

500

In [73]:
search_name_df.to_csv('data/search_name_df.csv', encoding='utf-8', index=False)

2. 검색 url 데이터프레임 생성

In [79]:
search_name_df = pd.read_csv('data/search_name_df.csv')
search_name_df.head(2)

,search_name
0,LOVE YOURSELF 結 ''Answer'' 방탄소년단
1,LOVE YOURSELF 轉 ''Tear'' 방탄소년단


In [84]:
search_name_list = list(search_name_df['search_name'])

In [91]:
# driver_path = 'C:/Users/Playdata/Desktop/dev/setup/chromdriver.exe'
driver_path = 'C:/Users/k9942/OneDrive/바탕 화면/playdata/dev/setup/chromdriver.exe'
driver = webdriver.Chrome(driver_path)

C:\Users\k9942\AppData\Local\Temp\ipykernel_9388\3454815965.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [92]:
# 앨범 검색 url 생성 함수
def create_searchAlbum_url(search_name):
    first_url = 'https://www.genie.co.kr/'
    driver.get(first_url)

    input_album_title = driver.find_elements(By.CSS_SELECTOR, 'input.ipt-search')[0]
    input_album_title.send_keys(search_name)

    button = driver.find_element(By.CSS_SELECTOR, 'input.btn-submit')
    button.click()
    
    now_url = driver.current_url
    search_url = now_url.replace('searchMain', 'searchAlbum')

    return search_url

In [93]:
searchAlbum_url_list = []

for search_name in search_name_list:
    searchAlbum_url = create_searchAlbum_url(search_name)
    searchAlbum_url_list.append(searchAlbum_url)
    time.sleep(5)

len(searchAlbum_url_list) # 500개 확인

500

In [103]:
total_searchAlbum_url_df = pd.DataFrame()
total_searchAlbum_url_df['url'] = searchAlbum_url_list

In [109]:
total_searchAlbum_url_df.to_csv('data/total_searchAlbum_url_df.csv', index = False, encoding='utf-8')

3. release_date 크롤링

In [216]:
total_searchAlbum_url_df = pd.read_csv('data/total_searchAlbum_url_df.csv')
total_searchAlbum_url_df.head(2)

,url
0,https://www.genie.co.kr/search/searchAlbum?que...
1,https://www.genie.co.kr/search/searchAlbum?que...


In [231]:
url = total_searchAlbum_url_df.loc[0][0]

In [239]:
url = 'https://www.genie.co.kr/search/searchAlbum?query=The%2520story%2520%25EA%25B0%2595%25EB%258B%25A4%25EB%258B%2588%25EC%2597%2598&Coll='

In [229]:
# driver_path = 'C:/Users/Playdata/Desktop/dev/setup/chromdriver.exe'
driver_path = 'C:/Users/k9942/OneDrive/바탕 화면/playdata/dev/setup/chromdriver.exe'
driver = webdriver.Chrome(driver_path)

C:\Users\k9942\AppData\Local\Temp\ipykernel_9388\3454815965.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [248]:
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [250]:
soup.select('dd.desc')

[]

In [251]:
release_date_list = []
for i in range(5):
    url = total_searchAlbum_url_df.loc[i][0]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    release_date = soup.select('dd.desc')
    
    if len(release_date) == 0:
        release_date = 'null'
    else:
        release_date = release_date[0].get_text().split('|')[0].strip()
    release_date_list.append(release_date)
    time.sleep(3)

In [247]:
url = 'https://www.genie.co.kr/search/searchAlbum?query=1%25C3%25B7%25CF%2587%253D1%2520%28UNDIVIDED%29%2520Wanna%2520One%2520%28%25EC%259B%258C%25EB%2584%2588%25EC%259B%2590%29'

In [252]:
release_date_list

['2018.08.24', '2018.05.18', '2018.11.02', '2018.03.19', 'null']